In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
df = pd.read_csv('Harvard_preprocessed.csv')

In [4]:
df.head()

,course_id,userid_DI,registered,viewed,explored,certified,grade,nevents,ndays_act,nchapters,nforum_posts,Age,LoE_DI_labels,gender_labels
0,HarvardX/CB22x/2013_Spring,MHxPC130442623,1,0,0,0,0.00,1,9,1,0,22,0,1
1,HarvardX/CB22x/2013_Spring,MHxPC130275857,1,0,0,0,0.00,1,16,1,0,23,0,1
2,HarvardX/CB22x/2013_Spring,MHxPC130539455,1,1,0,0,0.00,42,6,3,0,23,0,1
3,HarvardX/CB22x/2013_Spring,MHxPC130088379,1,1,0,0,0.00,70,3,3,0,23,0,1
4,HarvardX/CB22x/2013_Spring,MHxPC130024894,1,1,0,0,0.07,175,9,7,0,23,0,1


In [5]:
c=df.pop('grade')
df['grade']=c

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309638 entries, 0 to 309637
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   course_id      309638 non-null  object 
 1   userid_DI      309638 non-null  object 
 2   registered     309638 non-null  int64  
 3   viewed         309638 non-null  int64  
 4   explored       309638 non-null  int64  
 5   certified      309638 non-null  int64  
 6   nevents        309638 non-null  int64  
 7   ndays_act      309638 non-null  int64  
 8   nchapters      309638 non-null  int64  
 9   nforum_posts   309638 non-null  int64  
 10  Age            309638 non-null  int64  
 11  LoE_DI_labels  309638 non-null  int64  
 12  gender_labels  309638 non-null  int64  
 13  grade          309638 non-null  float64
dtypes: float64(1), int64(11), object(2)
memory usage: 33.1+ MB


In [7]:
df = df.drop(['course_id','userid_DI'],axis = 1)
df.head()

,registered,viewed,explored,certified,nevents,ndays_act,nchapters,nforum_posts,Age,LoE_DI_labels,gender_labels,grade
0,1,0,0,0,1,9,1,0,22,0,1,0.00
1,1,0,0,0,1,16,1,0,23,0,1,0.00
2,1,1,0,0,42,6,3,0,23,0,1,0.00
3,1,1,0,0,70,3,3,0,23,0,1,0.00
4,1,1,0,0,175,9,7,0,23,0,1,0.07


In [8]:
#Identify input and target columns
input_cols,target_cols=df.columns[:-1],df.columns[-1]
input_df,targets=df[input_cols].copy(),df[target_cols].copy()

In [9]:
cols1=input_df.columns[:9]

In [10]:
scaler=MinMaxScaler().fit(input_df[cols1])
input_df[cols1]=scaler.transform(input_df[cols1])

In [11]:
input_df.head()

,registered,viewed,explored,certified,nevents,ndays_act,nchapters,nforum_posts,Age,LoE_DI_labels,gender_labels
0,0.0,0.0,0.0,0.0,0.000000,0.045714,0.000000,0.0,0.268293,0,1
1,0.0,0.0,0.0,0.0,0.000000,0.085714,0.000000,0.0,0.280488,0,1
2,0.0,1.0,0.0,0.0,0.000934,0.028571,0.060606,0.0,0.280488,0,1
3,0.0,1.0,0.0,0.0,0.001573,0.011429,0.060606,0.0,0.280488,0,1
4,0.0,1.0,0.0,0.0,0.003965,0.045714,0.181818,0.0,0.280488,0,1


In [12]:
cols2=input_df.columns[9:]

In [13]:
#input_df[cols2]

In [13]:
#For categorical columns
one_hot_encoded_data = pd.get_dummies(input_df, columns = ['LoE_DI_labels', 'gender_labels'])
print(one_hot_encoded_data)

        registered  viewed  explored  certified   nevents  ndays_act  \
0              0.0     0.0       0.0        0.0  0.000000   0.045714   
1              0.0     0.0       0.0        0.0  0.000000   0.085714   
2              0.0     1.0       0.0        0.0  0.000934   0.028571   
3              0.0     1.0       0.0        0.0  0.001573   0.011429   
4              0.0     1.0       0.0        0.0  0.003965   0.045714   
...            ...     ...       ...        ...       ...        ...   
309633         0.0     1.0       0.0        0.0  0.000433   0.000000   
309634         0.0     1.0       0.0        0.0  0.001709   0.000000   
309635         0.0     0.0       0.0        0.0  0.000023   0.000000   
309636         0.0     1.0       0.0        0.0  0.000820   0.011429   
309637         0.0     0.0       0.0        0.0  0.000000   0.000000   

        nchapters  nforum_posts       Age  LoE_DI_labels_0  LoE_DI_labels_1  \
0        0.000000           0.0  0.268293               

In [14]:
input_df = one_hot_encoded_data

In [15]:
input_df.shape

(309638, 17)

In [17]:
#X_train,X_test,y_train,y_test=train_test_split(input_df,targets,test_size=0.25,random_state=42)

In [19]:
#X_train.head()

In [17]:
#y_train.head()

The first model we are implementing is the Linear regression model. Next, we will check SVM,Naive Bayes,Decision tree (and its variants i.e. ensemble techniques). Finally we will compare the performance of the models using MAE and MSE on the test set.

In [18]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import KFold,cross_val_score
import warnings
warnings.simplefilter(action='ignore')

In [19]:
scores= [] #to store r squared
mse_list= [] #to store RMSE
mae_list = []
lrmodel = LinearRegression()
cv = KFold(n_splits=20, random_state=42, shuffle=True)

In [20]:
#from sklearn.linear_model import LinearRegression

for train_index, test_index in cv.split(input_df):
    X_train, X_test, y_train, y_test = input_df.loc[train_index], input_df.loc[test_index], targets.loc[train_index], targets.loc[test_index]
    lrmodel.fit(X_train, y_train)
    y_predict = lrmodel.predict(X_test)
    scores.append(lrmodel.score(X_test, y_test))
    mse_fold = mean_squared_error(y_test, y_predict)
    mse_list.append(mse_fold)
    mae_fold = mean_absolute_error(y_test, y_predict)
    mae_list.append(mae_fold)


In [21]:

print('MSE',np.mean(mse_list))
print('MAE',np.mean(mae_list))
print('R-squared',np.mean(scores))

MSE 0.0016514216857952623
MAE 0.01072182928095386
R-squared 0.9069183407668178


In [22]:
#SVR
scores= [] #to store r squared
mse_list= [] #to store RMSE
mae_list = []
model2 = LinearSVR()
cv = KFold(n_splits=20, random_state=42, shuffle=True)
for train_index, test_index in cv.split(input_df):
    X_train, X_test, y_train, y_test = input_df.loc[train_index], input_df.loc[test_index], targets.loc[train_index], targets.loc[test_index]
    model2.fit(X_train, y_train)
    y_predict = model2.predict(X_test)
    scores.append(model2.score(X_test, y_test))
    mse_fold = mean_squared_error(y_test, y_predict)
    mse_list.append(mse_fold)
    mae_fold = mean_absolute_error(y_test, y_predict)
    mae_list.append(mae_fold)
print('MSE',np.mean(mse_list))
print('MAE',np.mean(mae_list))
print('R-squared',np.mean(scores))

MSE 0.0018783334992214743
MAE 0.007765247563378909
R-squared 0.8941222206070755


In [26]:
# Naive Bayes
'''
model3 = GaussianNB()
model3.fit(X_train, y_train)
y_pred = model3.predict(X_test)
print('MSE',mean_squared_error(y_pred,y_test))
print('MAE',mean_absolute_error(y_pred,y_test))
'''

"\nmodel3 = GaussianNB()\nmodel3.fit(X_train, y_train)\ny_pred = model3.predict(X_test)\nprint('MSE',mean_squared_error(y_pred,y_test))\nprint('MAE',mean_absolute_error(y_pred,y_test))\n"

In [23]:
scores= [] #to store r squared
mse_list= [] #to store RMSE
mae_list = []
model4=DecisionTreeRegressor()
cv = KFold(n_splits=20, random_state=42, shuffle=True)
for train_index, test_index in cv.split(input_df):
    X_train, X_test, y_train, y_test = input_df.loc[train_index], input_df.loc[test_index], targets.loc[train_index], targets.loc[test_index]
    model4.fit(X_train, y_train)
    y_predict = model4.predict(X_test)
    scores.append(model4.score(X_test, y_test))
    mse_fold = mean_squared_error(y_test, y_predict)
    mse_list.append(mse_fold)
    mae_fold = mean_absolute_error(y_test, y_predict)
    mae_list.append(mae_fold)
print('MSE',np.mean(mse_list))
print('MAE',np.mean(mae_list))
print('R-squared',np.mean(scores))

MSE 0.0017823395170229188
MAE 0.00739190913824665
R-squared 0.8994643516776202


In [24]:

scores= [] #to store r squared
mse_list= [] #to store RMSE
mae_list = []
model5=XGBRegressor()
cv = KFold(n_splits=20, random_state=42, shuffle=True)
for train_index, test_index in cv.split(input_df):
    X_train, X_test, y_train, y_test = input_df.loc[train_index], input_df.loc[test_index], targets.loc[train_index], targets.loc[test_index]
    model5.fit(X_train, y_train)
    y_predict = model5.predict(X_test)
    scores.append(model5.score(X_test, y_test))
    mse_fold = mean_squared_error(y_test, y_predict)
    mse_list.append(mse_fold)
    mae_fold = mean_absolute_error(y_test, y_predict)
    mae_list.append(mae_fold)
print('MSE',np.mean(mse_list))
print('MAE',np.mean(mae_list))
print('R-squared',np.mean(scores))

MSE 0.0008876220281619138
MAE 0.005679432054100453
R-squared 0.9499603642577933


In [25]:

scores= [] #to store r squared
mse_list= [] #to store RMSE
mae_list = []
model6=RandomForestRegressor()
cv = KFold(n_splits=20, random_state=42, shuffle=True)
for train_index, test_index in cv.split(input_df):
    X_train, X_test, y_train, y_test = input_df.loc[train_index], input_df.loc[test_index], targets.loc[train_index], targets.loc[test_index]
    model6.fit(X_train, y_train)
    y_predict = model6.predict(X_test)
    scores.append(model6.score(X_test, y_test))
    mse_fold = mean_squared_error(y_test, y_predict)
    mse_list.append(mse_fold)
    mae_fold = mean_absolute_error(y_test, y_predict)
    mae_list.append(mae_fold)
print('MSE',np.mean(mse_list))
print('MAE',np.mean(mae_list))
print('R-squared',np.mean(scores))

MSE 0.0009186260179721575
MAE 0.0057573677800639034
R-squared 0.9481921356088556


In [26]:

scores= [] #to store r squared
mse_list= [] #to store RMSE
mae_list = []
model7=AdaBoostRegressor()
cv = KFold(n_splits=20, random_state=42, shuffle=True)
for train_index, test_index in cv.split(input_df):
    X_train, X_test, y_train, y_test = input_df.loc[train_index], input_df.loc[test_index], targets.loc[train_index], targets.loc[test_index]
    model7.fit(X_train, y_train)
    y_predict = model7.predict(X_test)
    scores.append(model7.score(X_test, y_test))
    mse_fold = mean_squared_error(y_test, y_predict)
    mse_list.append(mse_fold)
    mae_fold = mean_absolute_error(y_test, y_predict)
    mae_list.append(mae_fold)
print('MSE',np.mean(mse_list))
print('MAE',np.mean(mae_list))
print('R-squared',np.mean(scores))

MSE 0.0018348119643949428
MAE 0.012225943720838192
R-squared 0.8964434518947083


In [27]:

scores= [] #to store r squared
mse_list= [] #to store RMSE
mae_list = []
model8=GradientBoostingRegressor()
cv = KFold(n_splits=20, random_state=42, shuffle=True)
for train_index, test_index in cv.split(input_df):
    X_train, X_test, y_train, y_test = input_df.loc[train_index], input_df.loc[test_index], targets.loc[train_index], targets.loc[test_index]
    model8.fit(X_train, y_train)
    y_predict = model8.predict(X_test)
    scores.append(model8.score(X_test, y_test))
    mse_fold = mean_squared_error(y_test, y_predict)
    mse_list.append(mse_fold)
    mae_fold = mean_absolute_error(y_test, y_predict)
    mae_list.append(mae_fold)
print('MSE',np.mean(mse_list))
print('MAE',np.mean(mae_list))
print('R-squared',np.mean(scores))

MSE 0.000917025108636396
MAE 0.0060004606331448395
R-squared 0.9483038139905293


In the above analysis, we can see that we get the best results using XGBoost Regressor.